In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.data import Field, BucketIterator

import random
import math
import time
from torchtext import data
from transformers import T5Tokenizer, T5Model
from torch.utils.tensorboard import SummaryWriter

In [2]:
writer = SummaryWriter('runs/marco_v2_120000')

In [3]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [4]:
tokens = tokenizer.tokenize('Hello world how are you?')

print(tokens)

['▁Hello', '▁world', '▁how', '▁are', '▁you', '?']


In [5]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[8774, 296, 149, 33, 25, 58]


In [6]:
init_token = tokenizer.pad_token
eos_token = tokenizer.eos_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

<pad> </s> <pad> <unk>


In [7]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

0 1 0 2


In [8]:
max_input_length = tokenizer.max_model_input_sizes['t5-small']

print(max_input_length)

512


In [9]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [10]:
SRC = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

TRG = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

In [11]:
fields = [('src', SRC), ('trg', TRG)]

In [12]:
train_data = data.TabularDataset.splits(
                path = '',
                train = 'marco2.csv',
                format = 'csv',
                fields = fields,
                skip_header = True)

train_data , valid_data = train_data[0].split(split_ratio=0.98,
                                             random_state = random.seed(4321))

In [13]:
print(len(train_data.examples))
print(len(valid_data.examples))

117600
2400


In [14]:
print(vars(train_data.examples[10000]))

{'src': [2625, 3, 10, 167, 19652, 7, 67, 1224, 44, 306, 7902, 5, 387, 24, 65, 118, 3, 25189, 21, 209, 1962, 19, 1346, 12, 3281, 227, 34, 65, 3, 22964, 5, 3, 99, 150, 119, 1573, 13, 387, 30929, 23, 106, 19, 347, 6, 182, 1312, 5011, 387, 164, 36, 1346, 12, 3281, 3, 99, 34, 65, 118, 16, 8, 5040, 21, 3, 9, 298, 5, 3, 9, 1196, 13, 10607, 33, 347, 45, 8515, 3253, 5, 11417, 3, 10, 149, 307, 12, 13374, 387, 21, 1346, 169], 'trg': [209, 1962]}


In [15]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6000])['src'])

print(tokens)
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6000])['trg'])

print(tokens)

['▁context', '▁', ':', '▁transition', 'al', '▁period', ':', '▁week', '▁two', '-', 'to', '-', 'four', '.', '▁the', '▁second', '▁week', '▁of', '▁life', '▁brings', '▁great', '▁changes', '▁for', '▁the', '▁puppy', '.', '▁ears', '▁and', '▁eyes', '▁sealed', '▁since', '▁birth', '▁begin', '▁to', '▁open', '▁during', '▁this', '▁period', ',', '▁ears', '▁at', '▁about', '▁two', '▁weeks', '▁and', '▁eye', 'lid', 's', '▁between', '▁', 'ten', '▁to', '▁16', '▁days', '.', '▁this', '▁gives', '▁the', '▁fur', 'ry', '▁babies', '▁', 'a', '▁new', '▁sense', '▁of', '▁their', '▁world', '.', '▁query', '▁', ':', '▁how', '▁old', '▁are', '▁puppies', '▁when', '▁their', '▁ears', '▁open']
['▁two', '▁weeks']


In [16]:
device = torch.device('cuda')

BATCH_SIZE = 20

train_iterator, valid_iterator = BucketIterator.splits(
                                (train_data, valid_data), 
                                batch_size = BATCH_SIZE,
                                device = device,
                                sort_key=lambda x: len(x.src))

In [17]:
class T5Network(nn.Module):
    def __init__(self):
        
        super().__init__()
        
        self.t5 = t5 = T5Model.from_pretrained('t5-small')
        
        self.out = nn.Linear(t5.config.to_dict()['d_model'], t5.config.to_dict()['vocab_size'])
                
    def forward(self, src, trg):
        
        embedded = self.t5(input_ids=src, decoder_input_ids=trg)
        
        output = self.out(embedded[0])
        
        return output

In [18]:
model = T5Network().cuda()

Some weights of T5Model were not initialized from the model checkpoint at t5-small and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 76,988,544 trainable parameters


In [20]:
LEARNING_RATE = 0.0004

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [21]:
criterion = nn.CrossEntropyLoss(ignore_index = pad_token_idx)

In [22]:
N_EPOCHS = 4
CLIP = 1

In [23]:
for epoch in range(N_EPOCHS):
    
    start = time.time()
    # TRAIN 
    #########################################################################
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(train_iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg[:,:-1])
        
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    train_loss = epoch_loss / len(train_iterator)
    #########################################################################
    
    # VALID
    #########################################################################
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(valid_iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg[:,:-1])
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    valid_loss = epoch_loss / len(valid_iterator)
    #########################################################################
    end = time.time()
    
    print(f"EPOCH : {epoch+1}\tTRAIN LOSS : {train_loss:.2f}\tVALID LOSS : {valid_loss:.2f}\tTIME : {end-start:.2f}\n")
    torch.save(model.state_dict(), f'model_{epoch+1}.pt')
    
    writer.add_scalar('TRAINING LOSS', train_loss, epoch+1)
    writer.add_scalar('VALIDATION LOSS', valid_loss, epoch+1)

EPOCH : 1	TRAIN LOSS : 1.69	VALID LOSS : 0.80	TIME : 1883.62

EPOCH : 2	TRAIN LOSS : 0.68	VALID LOSS : 0.70	TIME : 1749.28

EPOCH : 3	TRAIN LOSS : 0.55	VALID LOSS : 0.70	TIME : 2249.81

EPOCH : 4	TRAIN LOSS : 0.47	VALID LOSS : 0.71	TIME : 3021.30



In [24]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    model.eval()

    src_indexes = [init_token_idx] + sentence + [eos_token_idx]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    trg_indexes = [init_token_idx]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        
        with torch.no_grad():
            output = model(src_tensor, trg_tensor)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == eos_token_idx:
            break
            
    return trg_indexes[1:]

In [25]:
idxs = random.sample(range(0,len(valid_data.examples)),20)

for i in idxs:
    src = vars(valid_data.examples[i])['src']
    trg = vars(valid_data.examples[i])['trg']
    translation = translate_sentence(src, SRC, TRG, model, device)

    print(f"SRC : {' '.join(tokenizer.convert_ids_to_tokens(src))}")
    print(f"TRG : {' '.join(tokenizer.convert_ids_to_tokens(trg))}")
    print(f"PREDICTED : {' '.join(tokenizer.convert_ids_to_tokens(translation))}\n")

SRC : ▁context ▁ : ▁the ▁average ▁medical ▁transcription ist ▁salary ▁is ▁about ▁$40 k . ▁pay ▁is ▁ based ▁on ▁production , ▁so ▁the ▁more ▁productive ▁ a ▁transcription ist ▁is , ▁the ▁more ▁they ▁earn . ▁an ▁experienced ▁transcription ist , ▁with ▁high ▁production , ▁easily ▁earn s ▁$ 70 k ▁per ▁year . ▁query ▁ : ▁how ▁much ▁can ▁ a ▁transcription ist ▁make
TRG : ▁$40 k
PREDICTED : ▁$ 70 k ▁per ▁year . </s>

SRC : ▁context ▁ : ▁africa - the ▁second ▁largest ▁continent ▁in ▁the ▁world ▁is ▁also ▁home ▁to ▁the ▁largest ▁desert ▁in ▁the ▁world - the ▁ s a hara . in ▁fact ▁there ▁are ▁three ▁desert s ▁on ▁the ▁continent - the ▁ s a hara , ▁the ▁ n ami b ▁and ▁the ▁ kal a hari . f ric a - the ▁second ▁largest ▁continent ▁in ▁the ▁world ▁is ▁also ▁home ▁to ▁the ▁largest ▁desert ▁in ▁the ▁world - the ▁ s a hara . ▁query ▁ : ▁what ▁is ▁largest ▁africa n ▁desert
TRG : ▁ s a hara ▁desert
PREDICTED : ▁ s a hara </s>

SRC : ▁context ▁ : ▁overall , ▁red ▁onions ▁contain ▁ a ▁higher ▁amount ▁of ▁a